In [1]:
#Get all the imports
import jax
import os
import sys

jax.config.update('jax_default_prng_impl', 'unsafe_rbg')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["LIBTPU_INIT_ARGS"] = os.environ.get("LIBTPU_INIT_ARGS","") + " --xla_tpu_spmd_rng_bit_generator_unsafe=true"
print(f"Found {jax.device_count()} devices.")

from typing import Sequence
import datetime
from absl import app
from flax.linen import partitioning as nn_partitioning
from flax import linen as nn
import numpy as np
import optax
from tensorboardX import SummaryWriter

from layers import Transformer
import pyconfig
from input_pipeline import create_data_iterator_with_tokenizer
import max_utils
import checkpointing

import jax.numpy as jnp
from jax import random
from jax.sharding import PartitionSpec as P
from jax.sharding import Mesh

from jax.experimental.compilation_cache import compilation_cache as cc

from cloud_tpu_diagnostics import diagnostic
from cloud_tpu_diagnostics.configuration import debug_configuration
from cloud_tpu_diagnostics.configuration import diagnostic_configuration
from cloud_tpu_diagnostics.configuration import stack_trace_configuration

2023-10-06 23:22:40.493575: I external/xla/xla/pjrt/pjrt_api.cc:98] GetPjrtApi was found for tpu at /home/mazumdera/.local/lib/python3.10/site-packages/libtpu/libtpu.so
2023-10-06 23:22:40.493608: I external/xla/xla/pjrt/pjrt_api.cc:67] PJRT_Api is set for device type tpu
2023-10-06 23:22:40.493622: I external/xla/xla/pjrt/pjrt_api.cc:72] PJRT plugin for tpu has PJRT API version 0.32. The framework PJRT API version is 0.32.
2023-10-06 23:22:43.030272: I external/xla/xla/pjrt/pjrt_c_api_client.cc:119] PjRtCApiClient created.


Found 8 devices.


2023-10-06 23:22:44.767361: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 23:22:44.873879: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 23:22:44.874797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 23:22:46.312165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mazumdera/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#free up TPU if needed
# !sudo lsof -w /dev/accel*
# !pkill -9 python3

In [2]:
from flax.training import train_state

In [20]:
base_output_directory="base_output_directory=gs://mazumdera-test-bucket/maxtext/lg/10052023/3"
file_pattern_for_train_data="file_pattern_for_train_data=gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords"
file_pattern_for_eval_data="file_pattern_for_eval_data=gs://yejingxin-us-central2/external/lg/dummy-data/valid/*tfrecords"

In [38]:
# commandline_args = [MaxText/configs/base.yml run_name=1xv3-32 save_period=5 dcn_data_parallelism=1 ici_data_parallelism=2 ici_tensor_parallelism=4 steps=20 enable_profiler=true remat_policy=full base_emb_dim=4096 base_num_heads=64 head_dim=64 vocab_size=50272 base_num_decoder_layers=16 per_device_batch_size=0.5 file_pattern_for_train_data="gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords" file_pattern_for_eval_data="gs://yejingxin-us-central2/external/lg/dummy-data/valid/*tfrecords" enable_profiler=true base_output_directory="gs://mazumdera-test-bucket/maxtext/lg/10052023/3" dataset_type="lg" max_predict_length=512]
commandline_args = ["dummy", "configs/base.yml","run_name=1xv3-32", "dcn_data_parallelism=1", "save_period=5","ici_data_parallelism=2","ici_tensor_parallelism=4","steps=20","enable_profiler=true","remat_policy=full","base_emb_dim=4096","base_num_heads=64","head_dim=64","vocab_size=50272","base_num_decoder_layers=16","per_device_batch_size=0.5","enable_profiler=true",file_pattern_for_train_data, file_pattern_for_eval_data, base_output_directory,"dataset_type=\"lg\"","max_predict_length=512"]


In [39]:
pyconfig.initialize(commandline_args)
config = pyconfig.config

In [43]:
checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
      config.checkpoint_dir,
      config.enable_checkpointing,
      config.async_checkpointing,
      config.save_period,
  )

Creating checkpoint manager...


2023-10-07 00:02:57.325465: I external/xla/xla/pjrt/distributed/service.cc:119] Experimental coordination service is enabled.
2023-10-07 00:02:57.325714: I external/xla/xla/pjrt/distributed/service.cc:149] Jax service listening on 10.128.0.67:34913
2023-10-07 00:02:57.351657: I external/tsl/tsl/distributed_runtime/coordination/coordination_service.cc:553] /job:jax_worker/replica:0/task:0 has connected to coordination service. Incarnation: 744586964027380521
2023-10-07 00:02:57.351793: I external/tsl/tsl/distributed_runtime/coordination/coordination_service_agent.cc:304] Coordination agent has successfully connected.
2023-10-07 00:02:57.352055: I external/xla/xla/pjrt/distributed/client.cc:132] Connected to distributed JAX controller


Checkpoint manager created!


In [45]:
# Initial PRNG Keys
init_rng, nextrng = random.split(random.PRNGKey(config.init_weights_seed), 2)

In [40]:
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)] (num_devices: 8)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1)
   TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1)]]

 [[TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0)
   TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)
   TpuDevice(id=4

In [42]:
model = Transformer(config, mesh)

In [44]:
tx = optax.adamw(
      max_utils.create_learning_rate_schedule(config),
      b1=config.adam_b1,
      b2=config.adam_b2,
      eps=config.adam_eps,
      eps_root=config.adam_eps_root,
      weight_decay=config.adam_weight_decay,
  )

In [46]:
state, state_mesh_annotations = max_utils.setup_initial_state(model, tx, config, init_rng, mesh, checkpoint_manager)

restoring state from this run's directory latest step         15


I0000 00:00:1696637039.202622  203532 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1696637039.205933  208523 google_auth_provider.cc:179] Running on GCE, using service account 630405687483-compute@developer.gserviceaccount.com


In [49]:
import functools
import max_utils
init_train_state_partial = functools.partial(max_utils.init_train_state, model, tx,
                                               config)

In [52]:
#get the flax.training.train_state.TrainState
abstract_state = jax.eval_shape(init_train_state_partial, init_rng)

In [53]:
abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fdb79be68c0>
    mesh = Mesh(device_ids=array([[[0, 1, 2, 3]],
    
           [[6, 7, 4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(4096, 16, 8192), dtype=float32), names=('embed', 'layers', 'mlp'), mesh=None, rules=None)}, 'wo': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(8192, 16, 4096), dtype=float32), names=('mlp', 'layers', 'embed'), mesh=None, rules=None)}}, 'pre_self_attention_layer_norm': {'scale': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(4096, 16), dtype=float32), names=('embed', 'layers'), mesh=None, rules=None)}, 'relpos_bias': {'rel_embedding': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(64, 16, 32), dtype=float32), names=('heads', 'layers', 'relpos_b

In [54]:
state_logical_annotations = nn.get_partition_spec(abstract_state)

In [55]:
state_logical_annotations

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fdb79be68c0>
    mesh = Mesh(device_ids=array([[[0, 1, 2, 3]],
    
           [[6, 7, 4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': PartitionSpec('embed', 'layers', 'mlp')}, 'wo': {'kernel': PartitionSpec('mlp', 'layers', 'embed')}}, 'pre_self_attention_layer_norm': {'scale': PartitionSpec('embed', 'layers')}, 'relpos_bias': {'rel_embedding': PartitionSpec('heads', 'layers', 'relpos_buckets')}, 'self_attention': {'key': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'key_layer_norm': {'scale': PartitionSpec('heads', 'layers')}, 'out': {'kernel': PartitionSpec('heads', 'layers', 'kv', 'embed')}, 'query': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'query_layer_norm': {'scale': PartitionSpec('heads', 'layers')}, 'value': {'kernel': PartitionS

In [57]:
unboxed_abstract_state = max_utils.unbox_logicallypartioned_trainstate(abstract_state)

In [58]:
unboxed_abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fdb79be68c0>
    mesh = Mesh(device_ids=array([[[0, 1, 2, 3]],
    
           [[6, 7, 4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': ShapeDtypeStruct(shape=(4096, 16, 8192), dtype=float32)}, 'wo': {'kernel': ShapeDtypeStruct(shape=(8192, 16, 4096), dtype=float32)}}, 'pre_self_attention_layer_norm': {'scale': ShapeDtypeStruct(shape=(4096, 16), dtype=float32)}, 'relpos_bias': {'rel_embedding': ShapeDtypeStruct(shape=(64, 16, 32), dtype=float32)}, 'self_attention': {'key': {'kernel': ShapeDtypeStruct(shape=(4096, 16, 64, 64), dtype=float32)}, 'key_layer_norm': {'scale': ShapeDtypeStruct(shape=(64, 16), dtype=float32)}, 'out': {'kernel': ShapeDtypeStruct(shape=(64, 16, 64, 4096), dtype=float32)}, 'query': {'kernel': ShapeDtypeStruct(shape=(4096, 16, 64

In [59]:
#Load the Lingvo TF config

import importlib
